<a href="https://colab.research.google.com/github/ykitaguchi77/Laboratory_course/blob/master/MNIST%20using%20Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**PytorchでMNIST**

In [ ]:
# PytorchでMNIST
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms


# データの準備
mnist_train = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
mnist_test = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

# データローダーの定義
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=1000, shuffle=False)


# モデルの定義
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# モデルの学習と評価
model = Net()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(10):
    model.train()
    total_train_loss = 0
    total_train_correct = 0
    total_train = 0

    for data, target in train_loader:
        optimizer.zero_grad()

        #forward
        output = model(data)
        loss = loss_fn(output, target)
        total_train_loss += loss.item()

        #backward
        loss.backward()
        optimizer.step()

        #calculate train_accuracy
        _, predicted = torch.max(output.data, 1)
        total_train += target.size(0)
        total_train_correct += (predicted == target).sum().item()

    train_loss = total_train_loss / len(train_loader)
    train_acc = total_train_correct / total_train

    # テストデータでの評価
    model.eval()
    total_val_loss = 0
    total_val_correct = 0
    total_val = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            loss = loss_fn(output, target)
            total_val_loss += loss.item()

            _, predicted = torch.max(output.data, 1)
            total_val += target.size(0)
            total_val_correct += (predicted == target).sum().item()

    val_loss = total_val_loss / len(test_loader)
    val_acc = total_val_correct / total_val

    print(f'Epoch {epoch},\n'
          f'Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f},\n'
          f'Training Accuracy: {train_acc:.4f}, Validation Accuracy: {val_acc:.4f}, \n')

# モデルの評価
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)  # Ensure 'target' is treated as a tensor
        correct += (predicted == target).sum().item()

print('Accuracy:', correct / total)


Epoch 0, ,
Training Loss: 0.2860, Validation Loss: 0.0921,
Training Accuracy: 0.9129, Validation Accuracy: 0.9716, 

Epoch 1, ,
Training Loss: 0.0813, Validation Loss: 0.0533,
Training Accuracy: 0.9756, Validation Accuracy: 0.9831, 

Epoch 2, ,
Training Loss: 0.0569, Validation Loss: 0.0422,
Training Accuracy: 0.9822, Validation Accuracy: 0.9857, 

Epoch 3, ,
Training Loss: 0.0464, Validation Loss: 0.0425,
Training Accuracy: 0.9857, Validation Accuracy: 0.9866, 

Epoch 4, ,
Training Loss: 0.0383, Validation Loss: 0.0350,
Training Accuracy: 0.9876, Validation Accuracy: 0.9894, 

Epoch 5, ,
Training Loss: 0.0328, Validation Loss: 0.0300,
Training Accuracy: 0.9895, Validation Accuracy: 0.9911, 

Epoch 6, ,
Training Loss: 0.0285, Validation Loss: 0.0355,
Training Accuracy: 0.9909, Validation Accuracy: 0.9888, 

